In [1]:
from pathlib import Path
from tqdm import tqdm
import pandas as po
import numpy as np
import warnings
import nltk
import os
import re

In [2]:
from paths import get_path_q
from rank import get_ranks, get_preds, write_preds, run_scoring
from utils import get_df_explanations, get_questions, process_qn

In [3]:
np.random.seed(42)
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/devansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
SEP = "#" * 100 + "\n"
MODE_TRAIN = "train"
MODE_DEV = "dev"
MODE_TEST = "test"
use_recursive_tfidf = True
bert_output_dir = ''

In [5]:
path_data   = Path("../data/")
path_tables = path_data.joinpath("raw/tables")

In [6]:
path_q = get_path_q(path_data, 'dev')
path_q

PosixPath('../data/raw/questions/questions_dev.tsv')

In [7]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    df_exp = get_df_explanations(path_tables, path_data)

  0%|          | 0/9029 [00:00<?, ?it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 9029/9029 [00:00<00:00, 18221.97it/s]

Explanation df shape: (9029, 5)


In [8]:
path_q

PosixPath('../data/raw/questions/questions_dev.tsv')

In [9]:
df_exp

,uid,text,tokens,lemmas,embedding
0,461f-00e5-56fe-dd2c,degrees fahrenheit ( F ) is a imperial unit us...,"[degree, fahrenheit, (, f, ), be, a, imperial,...","[degree, fahrenheit, (, f, ), imperial, unit, ...",None
1,93f5-b0bb-73b5-8b40,degrees celsius ( C ) is a metric unit used fo...,"[degree, celsius, (, c, ), be, a, metric, unit...","[degree, celsius, (, c, ), metric, unit, use, ...",None
2,140c-77ad-c6a7-5b26,degrees kelvin ( K ) is an SI unit used for me...,"[degree, kelvin, (, k, ), be, a, si, unit, use...","[degree, kelvin, (, k, ), si, unit, use, measu...",None
3,4aff-d072-35cc-3ebe,liters ( L ) is a metric unit used for measuri...,"[liter, (, l, ), be, a, metric, unit, use, for...","[liter, (, l, ), metric, unit, use, measure, v...",None
4,833d-29e2-dde2-a9fa,milliliters ( mL ) is a metric unit used for m...,"[milliliter, (, ml, ), be, a, metric, unit, us...","[milliliter, (, ml, ), metric, unit, use, meas...",None
...,...,...,...,...,...
9024,3934-4dac-608b-34d1,rusty metal is sometimes contaminated,"[rusty, metal, be, sometimes, contaminate]","[rusty, metal, sometimes, contaminate]",None
9025,b022-f0ce-9d00-e900,renewable resources are often available for ma...,"[renewable, resource, be, often, available, fo...","[renewable, resource, often, available, many, ...",None
9026,fb44-c786-daa6-dc3b,A celestial body is any identifiable object in...,"[a, celestial, body, be, any, identifiable, ob...","[celestial, body, identifiable, object, space]",None
9027,0777-2c55-d67f-cb5f,coins are usually round,"[coin, be, usually, round]","[coin, usually, round]",None


In [10]:
#df_exp.text

In [11]:
uid2idx = {uid: idx for idx, uid in enumerate(df_exp.uid.tolist())}

In [12]:
df = get_questions(str(path_q), uid2idx, path_data)

100%|██████████| 496/496 [00:00<00:00, 9453.58it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883
(496, 23)



/home/devansh/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [13]:
ranks = get_ranks(df, df_exp, use_embed=False, use_recursive_tfidf=True)

Num canonical explanations: 9029
Num canonical explanations: 9029


  0%|          | 0/496 [00:00<?, ?it/s]

Num canonical explanations: 9029


100%|██████████| 496/496 [00:00<00:00, 25350.33it/s]


In [14]:
#k = 100

In [15]:
'''
flag = 0 #if change in question
prev = '' #prev question
l = []
j = 0      #number of explanations taken
for i in preds:
    ques = i.split('\t')[0]
    expl = i.split('\t')[1]
    if(prev != ques):
        l.append(i)
        j = 1
        prev=ques
    else:
        if(j<k):
            l.append(i)
        j = j+1
    '''

"\nflag = 0 #if change in question\nprev = '' #prev question\nl = []\nj = 0      #number of explanations taken\nfor i in preds:\n    ques = i.split('\t')[0]\n    expl = i.split('\t')[1]\n    if(prev != ques):\n        l.append(i)\n        j = 1\n        prev=ques\n    else:\n        if(j<k):\n            l.append(i)\n        j = j+1\n    "

In [16]:
#len(d['MDSA_2009_5_16'])

In [17]:
'''d={}
for i in l:
    ques = i.split('\t')[0]
    expl = i.split('\t')[1]
    if(ques in d.keys()):
        d[ques].append(expl)
    else:
        d[ques] = [expl]
        '''

"d={}\nfor i in l:\n    ques = i.split('\t')[0]\n    expl = i.split('\t')[1]\n    if(ques in d.keys()):\n        d[ques].append(expl)\n    else:\n        d[ques] = [expl]\n        "

In [18]:
#k = 100 #number of explanations to take for each question

In [19]:
def clip_preds(preds, k=100):
    flag = 0 #if change in question
    prev = '' #prev question
    l = []
    j = 0      #number of explanations taken
    for i in preds:
        ques = i.split('\t')[0]
        expl = i.split('\t')[1]
        if(prev != ques):
            l.append(i)
            j = 1
            prev=ques
        else:
            if(j<k):
                l.append(i)
            j = j+1
    return l

In [20]:
preds = get_preds(ranks, df, df_exp)

In [21]:
preds = clip_preds(preds)

In [22]:
# To run on train set
#mode = MODE_TRAIN
#process_qn(path_data, path_q, df_exp, df, use_recursive_tfidf, bert_output_dir, uid2idx, ranks, preds, mode)

In [23]:
# To run on dev set
mode = MODE_DEV
process_qn(path_data, path_q, df_exp, df, use_recursive_tfidf, bert_output_dir, uid2idx, ranks, preds, mode)

####################################################################################################
 Processsing dev data set
Mode: dev, predict file will be saved to: predict_dev.txt
####################################################################################################
 BERT output dir not provided, running TFIDF-only methods
####################################################################################################
 Ranking
####################################################################################################
 Writing predictions file
(31744, 3)
####################################################################################################
 Preparing rerank data
Saving rerank data to: df_scores_dev.csv
####################################################################################################
 Scoring
         role     MAP     total  count  occurences
0  BACKGROUND  0.0288   14.2795    496          58
1     CENTRAL  0.4509  223.6290

In [24]:
#To run on dev set
#mode = MODE_TEST
#process_qn(path_data, path_q, df_exp, df, use_recursive_tfidf, bert_output_dir, uid2idx, ranks, preds, mode)